In [10]:
CLIENT_ID = "27h_IMp2XwMceQ"
CLIENT_SECRET = "O9mU-pimDWuOnJNoZbBCuuWRGGQ"

In [11]:
USER_AGENT = "python:<quekai_cp10> (by /u/<quekai>)"

In [12]:
USERNAME = "<username>"
PASSWORD = "<password>"
import requests

In [13]:
def login(username, password):
    if password is None:
        password = getpass.getpass("Enter reddit password for username {}:".format(username))
    headers = {"User-Agent": USER_AGENT}
    client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
    post_data = {"grant_type": "password", "username":username, "password":password}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
    return response.json()
    

In [14]:
token = login(USERNAME, PASSWORD)

In [15]:
print(token)

{'token_type': 'bearer', 'scope': '*', 'access_token': '275570065758-RD7fU_OofhIjK2qrTFqtnIO1bfw', 'expires_in': 3600}


In [16]:
subreddit = "worldnews"

In [17]:
url = "https://oauth.reddit.com/r/{}".format(subreddit)

In [18]:
headers = {"Authorization": "bearer {}".format(token['access_token']), "User-Agent": USER_AGENT}

In [19]:
response = requests.get(url,headers=headers)

In [22]:
print(response)

<Response [200]>


In [23]:
result = response.json()

In [24]:
for story in result['data']['children']:
    print(story['data']['title'])

Woman who accused Indian politician of rape hit by truck after father dies in police station
A Canadian court on Monday ruled that it was "false, misleading and deceptive" to label wines made in Jewish settlements in the occupied West Bank as a "Product of Israel"
Disturbing Report Shows Environmental Activists Killed Each Week: 'A devastating tally released Monday counts 164 environmentalists killed for their efforts in 2018. And that number is probably an underestimation.'
U.S. Senate fails to override Trump veto of bill stopping Saudi weapons sales
Prison riot in Brazil leaves 16 inmates decapitated and dozens more killed
Rockstar Games has paid no UK corporation tax in 10 years
Trump aide submitted drafts of 2016 'America First' energy speech to UAE for edits, emails show
Accused 9/11 mastermind open to testimony against Saudi Arabia
Gigantic, mysterious radiation leak traced to facility in Russia
Taiwanese contribute helmets to Hong Kong protesters
Hundreds of reindeer starve to d

In [25]:
from time import sleep

In [30]:
def get_links(subreddit, token, n_pages=5):
    stories = []
    after = None
    for page_number in range(n_pages):
        headers = {"Authorization": "bearer {}".format(token['access_token']), "User-Agent": USER_AGENT}
        url = "https://oauth.reddit.com/r/{}?limit=100".format(subreddit)
        if after:
            url += "&after={}".format(after)
        response = requests.get(url,headers=headers)
        result = response.json()
        after = result['data']['after']
        sleep(2)
        stories.extend([(story['data']['title'], story['data']['url'], story['data']['score']) for story in result['data']['children']])
    return stories

In [31]:
stories = get_links("worldnews", token)

In [32]:
print(len(stories))

500


In [48]:
import os
import hashlib
import codecs
data_floder = "raw/"

In [49]:
number_errors = 0
for title, url, score in stories:
    output_filename = hashlib.md5(url.encode()).hexdigest()
    fullpath = os.path.join(data_floder, output_filename + ".txt")
    headers = {
    'User-Agent': 'Mozilla/4.0(compatible; MSIE 5.5; Windows NT)'
    }
    try:
        response = requests.get(url, headers=headers)
        data = response.text
        with codecs.open(fullpath, 'w', 'utf8') as outf:
            outf.write(data)
    except Exception as e:
        number_errors += 1
        print(e)

HTTPSConnectionPool(host='reliefweb.int', port=443): Max retries exceeded with url: /report/india/india-monsoon-floods-emergency-plan-action-epoa-dref-n-mdrin023 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))


In [50]:
filenames = [os.path.join(data_floder, filename) for filename in os.listdir(data_floder)]

In [52]:
text_output_folder = "textonly/"

In [60]:
from lxml import etree
import lxml

In [61]:
skip_node_types = ["script", "head", "style", etree.Comment]

In [77]:
def get_text_from_node(node):
    if len(node) == 0:
        if node.text and len(node.text) > 100:
            return node.text
        else:
            return ""
    else:
        results = (get_text_from_node(child) for child in node if child.tag not in skip_node_types)
        return "\n".join(r for r in results if len(r) > 1)
def get_text_from_file(filename):
    with codecs.open(filename, encoding='utf8') as inf:
        html_tree = etree.parse(inf, etree.HTMLParser())
    return get_text_from_node(html_tree.getroot())

In [78]:
for filename in os.listdir(data_floder):
    text = get_text_from_file(os.path.join(data_floder, filename))
    with codecs.open(os.path.join(text_output_folder, filename), 'w', 'utf8') as outf:
        outf.write(text)